This File is to run the TVDN algorithm on AD and Control data and save the results

In [1]:
import sys
sys.path.append("/home/huaqingj/MyResearch/TVDN-AD")

In [4]:
from pyTVDN import TVDNDetect
from pathlib import Path
from scipy.io import loadmat, savemat
import numpy as np
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
import os
from scipy import signal
from tqdm import tqdm
import pickle
%matplotlib inline

In [5]:
os.chdir("/home/huaqingj/MyResearch/TVDN-AD")
resDir = Path("./results")
dataDir = Path("./data")

In [6]:
filAD = "87ADs_before_filter1000.mat"
filCtrl = "70Ctrls_before_filter1000.mat"
filADadd = "DK_timecourse.mat"

In [7]:
smADIdxs = [
    20,  
    42, 
    44, 
    45, 
    47, 
    48, 
    57, 
    58, 
    61, 
    62, 
    63, 
    65, 
    81, 
    82, 
    83, 
    87, 
]

smADIdxs = np.array(smADIdxs)- 1

In [8]:
smCtrlIdxs = [
    5,  
    6, 
    7, 
    8, 
    9, 
    10, 
    11, 
    12, 
    13, 
    14, 
    16, 
    17, 
    19, 
    21, 
    22, 
    24, 
    25, 
    26, 
    29, 
    30, 
    32, 
    33, 
    35, 
    38, 
    39, 
    57, 
    59, 
    60, 
    61, 
    62, 
    63, 
    64, 
    66, 
    67, 
    68, 
    69, 
    70, 
]
smCtrlIdxs = np.array(smCtrlIdxs) - 1

## Run data with TVDN

### Load the datasets

Note we have addtional data for AD

In [7]:
ADdatasets = loadmat(dataDir/filAD)["dk10"]
Ctrldatasets = loadmat(dataDir/filCtrl)["dk10"]

In [8]:
ADdatAdd = loadmat(dataDir/filADadd)["DK_timecourse"]

### Parameters for TVDN

In [9]:
paras = edict()
paras.Lmin = 200
paras.wh = 10
paras.MaxM = 20
paras.fct = 0.5
paras.r = None
paras.lamb = 1e-4
paras.T = 2
paras.is_detrend = True
paras.decimateRate = 5
paras.kappa = 3.005
paras.downRate = 20

if paras.decimateRate is None:
    paras.freq = 600
else:
    paras.freq = int(600/paras.decimateRate)

### Run Kappa tuning algorithm 

In [10]:
kps = np.linspace(1, 4, 1000)

In [10]:
for idx in tqdm(range(ADdatasets.shape[0])):
    ADdataset = ADdatasets[idx, :, :]
    detADa = []
    for i in range(5):
        detADt = TVDNDetect(Ymat=ADdataset[:, i::5], dataType="MEG", 
                             saveDir=None, 
                             showProgress=False, 
                             fName="demoMEG",
                             MaxM=paras.MaxM,
                             Lmin=paras.Lmin, 
                             lamb=paras.lamb, 
                             kappa=paras.kappa, 
                             freq=paras.freq,
                             r=paras.r,
                             T=paras.T,
                             is_detrend=paras.is_detrend,
                             decimateRate=None, 
                             downRate=paras.downRate)
        detADt.Screening(wh=paras.wh)
        detADt()
        detADt.TuningKappa(kps)
        detADa.append(detADt)
        
    saveFil = f"AD_data_det_{idx}_lamb{paras.lamb:.1E}_decimate{paras.decimateRate:.0f}_tuning.pkl"
    with open(resDir/saveFil, "wb") as f:
         pickle.dump(detADa, f)

100%|██████████| 87/87 [3:22:12<00:00, 139.45s/it]  


In [11]:
for idx in tqdm(range(Ctrldatasets.shape[0])):
    Ctrldataset = Ctrldatasets[idx, :, :]
    detCa = []
    for i in range(5):
        detCt = TVDNDetect(Ymat=Ctrldataset[:, i::5], dataType="MEG", 
                             saveDir=None, 
                             showProgress=False, 
                             fName="demoMEG",
                             MaxM=paras.MaxM,
                             Lmin=paras.Lmin, 
                             lamb=paras.lamb, 
                             kappa=paras.kappa, 
                             freq=paras.freq,
                             r=paras.r,
                             T=paras.T,
                             is_detrend=paras.is_detrend,
                             decimateRate=None, 
                             downRate=paras.downRate)
        detCt.Screening(wh=paras.wh)
        detCt()
        detCt.TuningKappa(kps)
        detCa.append(detCt)
        
    saveFil = f"Ctrl_data_det_{idx}_lamb{paras.lamb:.1E}_decimate{paras.decimateRate:.0f}_tuning.pkl"
    with open(resDir/saveFil, "wb") as f:
        pickle.dump(detCa, f)

100%|██████████| 70/70 [2:31:36<00:00, 129.95s/it]  


### Run Main TVDN without tuning kappa

$\kappa$ will be tuned another file

In [12]:
for idx in tqdm(range(ADdatasets.shape[0])):
    ADdataset = ADdatasets[idx, :, :]
    detAD = TVDNDetect(Ymat=ADdataset, dataType="MEG", 
                         saveDir=None, 
                         showProgress=False, 
                         fName="demoMEG",
                         MaxM=paras.MaxM,
                         Lmin=paras.Lmin, 
                         lamb=paras.lamb, 
                         kappa=paras.kappa, 
                         freq=paras.freq,
                         r=paras.r,
                         T=paras.T,
                         is_detrend=paras.is_detrend,
                         decimateRate=paras.decimateRate, 
                         downRate=paras.downRate)
    detAD.Screening(wh=paras.wh)
    detAD()
    detAD.TuningKappa(kps)
    
    saveFil = f"AD_data_det_{idx}_lamb{paras.lamb:.1E}_decimate{paras.decimateRate:.0f}.pkl"
    with open(resDir/saveFil, "wb") as f:
        pickle.dump(detAD, f)

  0%|          | 0/87 [00:00<?, ?it/s]R[write to console]: 
Attaching package: ‘signal’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, poly


100%|██████████| 87/87 [43:39<00:00, 30.10s/it]  


In [13]:
idx = 88
detAD = TVDNDetect(Ymat=ADdatAdd, dataType="MEG", 
                         saveDir=None, 
                         showProgress=False, 
                         fName="demoMEG",
                         MaxM=paras.MaxM,
                         Lmin=paras.Lmin, 
                         lamb=paras.lamb, 
                         kappa=paras.kappa, 
                         freq=paras.freq,
                         r=paras.r,
                         T=paras.T,
                         is_detrend=paras.is_detrend,
                         decimateRate=paras.decimateRate, 
                         downRate=paras.downRate)
detAD.Screening(wh=paras.wh)
detAD()
detAD.TuningKappa(kps)
    
saveFil = f"AD_data_det_{idx}_lamb{paras.lamb:.1E}_decimate{paras.decimateRate:.0f}.pkl"
with open(resDir/saveFil, "wb") as f:
    pickle.dump(detAD, f)

In [14]:
for idx in tqdm(range(Ctrldatasets.shape[0])):
    Ctrldataset = Ctrldatasets[idx, :, :]
    detC = TVDNDetect(Ymat=Ctrldataset, dataType="MEG", 
                         saveDir=None, 
                         showProgress=False, 
                         fName="demoMEG",
                         MaxM=paras.MaxM,
                         Lmin=paras.Lmin, 
                         lamb=paras.lamb, 
                         kappa=paras.kappa, 
                         freq=paras.freq,
                         r=paras.r,
                         T=paras.T,
                         is_detrend=paras.is_detrend,
                         decimateRate=paras.decimateRate, 
                         downRate=paras.downRate)
    detC.Screening(wh=paras.wh)
    detC()
    detC.TuningKappa(kps)
    
    saveFil = f"Ctrl_data_det_{idx}_lamb{paras.lamb:.1E}_decimate{paras.decimateRate:.0f}.pkl"
    with open(resDir/saveFil, "wb") as f:
        pickle.dump(detC, f)

100%|██████████| 70/70 [31:10<00:00, 26.73s/it]


### Save results

Note here I do not consider the order of the dataset, 
in fact for this part, order is not important.

In [10]:
detObjsNum = []
for fil in tqdm(list(resDir.glob("AD*04_decimate5_tuning.pkl"))):
    with open(fil, "rb") as f:
        dets = pickle.load(f)
    detObjsNum.append([det.numchgs for det in dets])
    
filName = f"AD_data_lamb{dets[0].paras.lamb:.1E}_tuningNum.pkl"
with open(resDir/filName, "wb") as f:
    pickle.dump(detObjsNum, f)

100%|██████████| 87/87 [10:39<00:00,  7.35s/it]


In [17]:
detObjsNumC = []
for fil in tqdm(list(resDir.glob("Ctrl*04_decimate5_tuning.pkl"))):
    
    with open(fil, "rb") as f:
        dets = pickle.load(f)
    detObjsNumC.append([det.numchgs for det in dets])
    
filName = f"Ctrl_data_lamb{dets[0].paras.lamb:.1E}_tuningNum.pkl"
with open(resDir/filName, "wb") as f:
    pickle.dump(detObjsNumC, f)

100%|██████████| 70/70 [08:07<00:00,  6.97s/it]


In [15]:
numchgss = []
for fil in tqdm(list(resDir.glob("AD*04_decimate5.pkl"))):
    with open(fil, "rb") as f:
        det = pickle.load(f)
    numchgss.append(det.numchgs)
    
filName = f"AD_data_lamb{det.paras.lamb:.1E}_Num.pkl"
with open(resDir/filName, "wb") as f:
    pickle.dump(numchgss, f)

100%|██████████| 88/88 [02:13<00:00,  1.52s/it]


In [16]:
numchgss = []
for fil in tqdm(list(resDir.glob("Ctrl*04_decimate5.pkl"))):
    with open(fil, "rb") as f:
        det = pickle.load(f)
    numchgss.append(det.numchgs)
    
filName = f"Ctrl_data_lamb{det.paras.lamb:.1E}_Num.pkl"
with open(resDir/filName, "wb") as f:
    pickle.dump(numchgss, f)

100%|██████████| 70/70 [01:49<00:00,  1.56s/it]
